In [1]:
import numpy as np
import pandas as pd
from pandas import pivot_table
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
borig = pd.read_csv('2018clean3.csv', header=0, index_col=0, dtype={'MainSic': object})

print(borig.shape)
borig.head(2)

(10772, 29)


,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted,Post,Val,MainSic,MainSection
0,"BRYANSTON SCHOOL,INCORPORATED","Bryanston House,\r\r\r\r\nBlandford,\r\r\r\r\n...",00226143,85310,19.6,32.5,NaN,NaN,0.0,0.0,...,Michael McGovern (Chief Operating Officer),500 to 999,"""Bryanston School"",Incorporated",False,05/04/2019 00:00:00,03/04/2019 13:43:34,DT11 0PX,True,85310,Education
1,"RED BAND CHEMICAL COMPANY, LIMITED","19 Smith's Place,\r\r\r\r\nLeith Walk,\r\r\r\r...",SC016876,47730,7.7,-4.2,23.7,-75.2,22.2,82.0,...,Philip Galt (Managing Director),250 to 499,"""RED BAND"" CHEMICAL COMPANY, LIMITED",False,05/04/2019 00:00:00,01/04/2019 12:37:21,EH6 8NU,True,47730,Wholesale and retail trade; repair of motor ve...


In [3]:
addnuts = pd.read_csv('pc2018_uk_NUTS-2016_v2.0.csv', sep=';')

print(addnuts.shape)
addnuts.head()

(1759911, 2)


,NUTS3,CODE
0,'UKM73','EH10 7DP'
1,'UKM73','EH10 7DX'
2,'UKM73','EH10 7DY'
3,'UKM73','EH10 7DZ'
4,'UKM73','EH10 7EA'


In [4]:
def remove_quotes(x):
    return str(x)[1:-1]

def ret_nuts2(x):
    return str(x)[:-1]

In [5]:
addnuts['NUTS3'] = addnuts['NUTS3'].apply(remove_quotes)
addnuts['CODE'] = addnuts['CODE'].apply(remove_quotes)
addnuts['NUTS3'] = addnuts['NUTS3'].apply(ret_nuts2)

addnuts.head(2)

addnuts.head(2)

,NUTS3,CODE
0,UKM7,EH10 7DP
1,UKM7,EH10 7DX


In [6]:
addnuts = addnuts[['CODE', 'NUTS3']]
addnuts.rename(columns={"CODE": "Post", "NUTS3": "Nuts2"}, inplace=True)

addnuts.head(2)

,Post,Nuts2
0,EH10 7DP,UKM7
1,EH10 7DX,UKM7


In [7]:
borig = pd.merge(borig, addnuts, on='Post')

borig.sample(2)

,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted,Post,Val,MainSic,MainSection,Nuts2
3881,MALMAISON TRADING LIMITED,"3rd Floor 95 Cromwell Road,\r\r\r\r\nLondon,\r...",08960725,55100,2.8,4.4,18.9,-25.0,41.0,48.0,...,1000 to 4999,MALMAISON TRADING LIMITED,False,05/04/2019 00:00:00,03/04/2019 09:27:42,SW7 4DL,True,55100,Accommodation and food service activities,UKI3
8934,ST. MARY'S SCHOOL (CALNE),"St Mary's School, Curzon Street,\r\r\r\r\nCaln...",00235572,85590,15.1,62.2,NaN,NaN,0.0,0.6,...,250 to 499,ST. MARY'S SCHOOL (CALNE),False,05/04/2019 00:00:00,01/04/2019 14:56:11,SN11 0DF,True,85590,Education,UKK1


In [8]:
n_piv = pivot_table(borig, values='DiffMeanHourlyPercent',
                     index=['Nuts2'], aggfunc=np.mean)


n_piv.rename(columns={'DiffMeanHourlyPercent':'Diff_avg'}, inplace=True)
n_piv.sort_values(by=['Diff_avg'], ascending=False, inplace=True)
n_piv['Diff_avg'] = round(n_piv['Diff_avg'], 2)
n_piv.reset_index(inplace=True)

n_piv.head(5)

,Nuts2,Diff_avg
0,UKM5,17.97
1,UKI3,17.05
2,UKI6,16.65
3,UKH3,15.97
4,UKI4,15.82


In [9]:
nuts_name = pd.read_csv('NUTS_Level_2_January_2018_Ultra_Generalised_Clipped_Boundaries_in_the_United_Kingdom.csv')
print(nuts_name.shape)
nuts_name.head(2)

(41, 9)


,objectid,nuts218cd,nuts218nm,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,UKC1,Tees Valley and Durham,414643,532242,-1.77440,54.685020,3.037020e+09,362943.953924
1,2,UKC2,Northumberland and Tyne and Wear,398898,602530,-2.01892,55.316841,5.572919e+09,487561.244740


In [10]:
nuts_name = nuts_name[['nuts218cd','nuts218nm','lat','long']]
nuts_name.rename(columns={'nuts218cd':'Nuts2','nuts218nm':'Name','long':'lon'}, inplace=True)

nuts_name.head()

,Nuts2,Name,lat,lon
0,UKC1,Tees Valley and Durham,54.685020,-1.77440
1,UKC2,Northumberland and Tyne and Wear,55.316841,-2.01892
2,UKD4,Lancashire,53.864182,-2.55392
3,UKD6,Cheshire,53.214230,-2.46927
4,UKD7,Merseyside,53.405350,-2.83893


In [11]:
n_piv = pd.merge(n_piv, nuts_name, on='Nuts2')

n_piv.tail(7)

,Nuts2,Diff_avg,Name,lat,lon
34,UKI5,11.32,Outer London - East and North East,51.588650,0.11095
35,UKI7,11.14,Outer London - West and North West,51.543549,-0.32033
36,UKL2,10.28,East Wales,52.384220,-3.28946
37,UKM8,10.28,West Central Scotland,56.027729,-4.63896
38,UKL1,9.27,West Wales,52.005711,-4.38131
39,UKM9,9.13,Southern Scotland,55.295059,-3.70478
40,UKN0,2.83,Northern Ireland,54.614941,-6.85481


In [12]:
# exclude Wales, Scotland and N Ireland due to no geojson data
print(f'n_piv length with Wales, Scotland and N Ireland: {len(n_piv)}')

n_piv = n_piv[n_piv['Nuts2'].str.contains('UKL') == False]
n_piv = n_piv[n_piv['Nuts2'].str.contains('UKM') == False]
n_piv = n_piv[n_piv['Nuts2'].str.contains('UKN') == False]
n_piv.reset_index(inplace=True)

print(f'n_piv length without Wales, Scotland and N Ireland: {len(n_piv)}')

n_piv length with Wales, Scotland and N Ireland: 41
n_piv length without Wales, Scotland and N Ireland: 33


In [13]:
n_piv.head(2)

,index,Nuts2,Diff_avg,Name,lat,lon
0,1,UKI3,17.05,Inner London - West,51.518639,-0.16383
1,2,UKI6,16.65,Outer London - South,51.365608,-0.04977


In [14]:
# n_piv.to_csv('2018countyavg.csv')

In [16]:
# sorted(n_piv['Nuts2'].unique())

In [ ]:
# this concludes pt 3.7